In [1]:
!pip install medmnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=18044b00a406d07d20688d6bce3dfc1b5113e77a8db2f12e2e46cab46876a976
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/
%mkdir SGAN_OUTPUT

MessageError: ignored

In [ ]:
from tqdm import tqdm
import numpy
import medmnist
import numpy as np
import torch.utils.data as data
from medmnist import INFO

def build_datasets(dataset):
  images = []
  labels = []

  for image in tqdm(dataset):
      try:
        r = numpy.array(numpy.array(image[0])[:,:,0])
        g = numpy.array(numpy.array(image[0])[:,:,1])
        b = numpy.array(numpy.array(image[0])[:,:,2])
        pixel = (r + g + b)/3
      except:
        pixel = numpy.array(numpy.array(image[0])[:,:])

      images.append(pixel) # uses only one colorchannel atm
      labels.append(int(image[1]))

  return (numpy.array(images), numpy.array(labels))

def load_data(data_flag="OrganAMNIST".lower(), download = True):
    '''
    Loads and transforms specified data set with data augmentation (random vertical and horizontal flips)
    '''
    info = INFO[data_flag]
    task = info['task']
    n_channels = info['n_channels']
    n_classes = len(info['label'])

    DataClass = getattr(medmnist, info['python_class'])

    # load the data
    train_dataset = DataClass(split='train', download=download)
    test_dataset = DataClass(split='test', download=download)

    return build_datasets(train_dataset), build_datasets(test_dataset), n_classes


_, _, n_classes =  load_data()
del(_)

Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz


100%|██████████| 17778/17778 [00:01<00:00, 14808.18it/s]


In [ ]:
# example of semi-supervised gan for mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend

# custom activation function
def custom_activation(output):
	logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
	result = logexpsum / (logexpsum + 1.0)
	return result

# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(28,28,1), n_classes=n_classes):
	# image input
	in_image = Input(shape=in_shape)
	# downsample
	fe = Conv2D(128*8, (3,3), strides=(2,2), padding='same')(in_image)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(128*8, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(128*8, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output layer nodes
	fe = Dense(n_classes)(fe)
	# supervised output
	c_out_layer = Activation('softmax')(fe)
	# define and compile supervised discriminator model
	c_model = Model(in_image, c_out_layer)
	c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
	# unsupervised output
	d_out_layer = Lambda(custom_activation)(fe)
	# define and compile unsupervised discriminator model
	d_model = Model(in_image, d_out_layer)
	d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00001))
	return d_model, c_model

# define the standalone generator model
def define_generator(latent_dim):
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 128*8 * 7 * 7
	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((7, 7, 128*8))(gen)
	# upsample to 14x14
	gen = Conv2DTranspose(128*8, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 28x28
	gen = Conv2DTranspose(128*8, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
	# define model
	model = Model(in_lat, out_layer)
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect image output from generator as input to discriminator
	gan_output = d_model(g_model.output)
	# define gan model as taking noise and outputting a classification
	model = Model(g_model.input, gan_output)
	# compile model
	opt = Adam(lr=0.00001)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load the images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _), n_classes = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return [X, trainy]

# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=n_classes):
	X, y = dataset
	X_list, y_list = list(), list()
	n_per_class = int(n_samples / n_classes)
	for i in range(n_classes):
		# get all images for this class
		X_with_class = X[y == i]
		# choose random instances
		ix = randint(0, len(X_with_class), n_per_class)
		# add to list
		[X_list.append(X_with_class[j]) for j in ix]
		[y_list.append(i) for j in ix]
	return asarray(X_list), asarray(y_list)

# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	z_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = z_input.reshape(n_samples, latent_dim)
	return z_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict(z_input)
	# create class labels
	y = zeros((n_samples, 1))
	return images, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(100):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = './SGAN_OUTPUT/generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# evaluate the classifier model
	X, y = dataset
	_, acc = c_model.evaluate(X, y, verbose=0)
	print('Classifier Accuracy: %.3f%%' % (acc * 100))
	# save the generator model
	filename2 = './SGAN_OUTPUT/g_model_%04d.h5' % (step+1)
	g_model.save(filename2)
	# save the classifier model
	filename3 = './SGAN_OUTPUT/c_model_%04d.h5' % (step+1)
	c_model.save(filename3)
	print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=5, n_classes=n_classes):
	# select supervised dataset
	X_sup, y_sup = select_supervised_samples(dataset, n_classes)
	print(X_sup.shape, y_sup.shape)
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps), "\n"*3)
	# manually enumerate epochs
	for i in range(n_steps):
		# update supervised discriminator (c)
		[Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
		c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
		# update unsupervised discriminator (d)
		[X_real, _], y_real = generate_real_samples(dataset, half_batch)
		d_loss1 = d_model.train_on_batch(X_real, y_real)
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		d_loss2 = d_model.train_on_batch(X_fake, y_fake)
		# update generator (g)
		X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
		# evaluate the model performance every so often
		if (i+1) % 1000 == 0:
			summarize_performance(i, g_model, c_model, latent_dim, dataset)

# size of the latent space
latent_dim = 128
# create the discriminator models
d_model, c_model = define_discriminator(n_classes=n_classes)
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_classes=n_classes)

Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz


100%|██████████| 17778/17778 [00:02<00:00, 8218.75it/s]


(11, 28, 28, 1) (11,)
n_epochs=200, n_batch=5, 1/2=2, b/e=6916, steps=1383200 



1/1 [==============================] - 0s 156ms/step
>1, c[2.390,50], d[0.087,2.485], g[0.087]
1/1 [==============================] - 0s 30ms/step
>2, c[2.411,0], d[0.087,2.485], g[0.087]
1/1 [==============================] - 0s 21ms/step
>3, c[2.406,0], d[0.087,2.485], g[0.087]
1/1 [==============================] - 0s 27ms/step
>4, c[2.422,0], d[0.086,2.485], g[0.087]
1/1 [==============================] - 0s 25ms/step
>5, c[2.399,0], d[0.087,2.485], g[0.087]
1/1 [==============================] - 0s 44ms/step
>6, c[2.372,50], d[0.086,2.485], g[0.087]
1/1 [==============================] - 0s 21ms/step
>7, c[2.359,50], d[0.085,2.485], g[0.087]
1/1 [==============================] - 0s 25ms/step
>8, c[2.353,100], d[0.084,2.485], g[0.087]
1/1 [==============================] - 0s 44ms/step
>9, c[2.355,100], d[0.084,2.485], g[0.087]
1/1 [==============================] - 0s 21ms/step
>10, c[2.372,50], d[0

Classifier Accuracy: 22.677%
>Saved: ./SGAN_OUTPUT/generated_plot_1000.png, ./SGAN_OUTPUT/g_model_1000.h5, and ./SGAN_OUTPUT/c_model_1000.h5
1/1 [==============================] - 0s 27ms/step
>1001, c[0.002,100], d[0.125,0.696], g[0.671]
1/1 [==============================] - 0s 28ms/step
>1002, c[0.001,100], d[0.009,0.579], g[1.076]
1/1 [==============================] - 0s 25ms/step
>1003, c[0.004,100], d[0.470,0.677], g[1.168]
1/1 [==============================] - 0s 29ms/step
>1004, c[0.012,100], d[0.008,0.852], g[1.118]
1/1 [==============================] - 0s 27ms/step
>1005, c[0.001,100], d[0.056,0.484], g[1.014]
1/1 [==============================] - 0s 26ms/step
>1006, c[0.000,100], d[0.278,0.257], g[1.830]
1/1 [==============================] - 0s 28ms/step
>1007, c[0.000,100], d[0.010,0.194], g[1.736]
1/1 [==============================] - 0s 28ms/step
>1008, c[0.018,100], d[0.005,0.259], g[2.116]
1/1 [==============================] - 0s 26ms/step
>1009, c[0.003,100], d[

Classifier Accuracy: 22.518%
>Saved: ./SGAN_OUTPUT/generated_plot_2000.png, ./SGAN_OUTPUT/g_model_2000.h5, and ./SGAN_OUTPUT/c_model_2000.h5
1/1 [==============================] - 0s 27ms/step
>2001, c[0.018,100], d[0.155,0.113], g[2.239]
1/1 [==============================] - 0s 26ms/step
>2002, c[0.002,100], d[0.902,0.123], g[2.571]
1/1 [==============================] - 0s 26ms/step
>2003, c[0.000,100], d[0.385,0.073], g[2.519]
1/1 [==============================] - 0s 27ms/step
>2004, c[0.000,100], d[0.298,0.048], g[2.314]
1/1 [==============================] - 0s 28ms/step
>2005, c[0.012,100], d[1.200,0.065], g[2.600]
1/1 [==============================] - 0s 28ms/step
>2006, c[0.000,100], d[0.105,0.140], g[2.503]
1/1 [==============================] - 0s 27ms/step
>2007, c[0.013,100], d[0.387,0.095], g[2.456]
1/1 [==============================] - 0s 27ms/step
>2008, c[0.001,100], d[0.122,0.106], g[2.331]
1/1 [==============================] - 0s 28ms/step
>2009, c[0.000,100], d[

Classifier Accuracy: 29.797%
>Saved: ./SGAN_OUTPUT/generated_plot_3000.png, ./SGAN_OUTPUT/g_model_3000.h5, and ./SGAN_OUTPUT/c_model_3000.h5
1/1 [==============================] - 0s 22ms/step
>3001, c[0.001,100], d[0.079,0.101], g[1.923]
1/1 [==============================] - 0s 21ms/step
>3002, c[0.002,100], d[0.168,0.107], g[2.033]
1/1 [==============================] - 0s 19ms/step
>3003, c[0.002,100], d[1.168,0.487], g[1.510]
1/1 [==============================] - 0s 22ms/step
>3004, c[0.001,100], d[0.255,0.313], g[1.923]
1/1 [==============================] - 0s 29ms/step
>3005, c[0.010,100], d[0.331,0.309], g[1.130]
1/1 [==============================] - 0s 28ms/step
>3006, c[0.003,100], d[0.346,1.184], g[1.441]
1/1 [==============================] - 0s 27ms/step
>3007, c[0.004,100], d[0.959,0.262], g[1.091]
1/1 [==============================] - 0s 26ms/step
>3008, c[0.013,100], d[0.465,0.469], g[1.289]
1/1 [==============================] - 0s 26ms/step
>3009, c[0.000,100], d[

Classifier Accuracy: 26.911%
>Saved: ./SGAN_OUTPUT/generated_plot_4000.png, ./SGAN_OUTPUT/g_model_4000.h5, and ./SGAN_OUTPUT/c_model_4000.h5
1/1 [==============================] - 0s 27ms/step
>4001, c[0.005,100], d[0.620,0.893], g[1.459]
1/1 [==============================] - 0s 31ms/step
>4002, c[0.002,100], d[0.723,0.277], g[0.904]
1/1 [==============================] - 0s 28ms/step
>4003, c[0.000,100], d[0.240,0.407], g[1.415]
1/1 [==============================] - 0s 26ms/step
>4004, c[0.004,100], d[0.347,1.158], g[0.872]
1/1 [==============================] - 0s 28ms/step
>4005, c[0.002,100], d[0.296,0.267], g[1.327]
1/1 [==============================] - 0s 30ms/step
>4006, c[0.002,100], d[0.258,0.308], g[1.087]
1/1 [==============================] - 0s 26ms/step
>4007, c[0.001,100], d[0.263,0.372], g[0.901]
1/1 [==============================] - 0s 26ms/step
>4008, c[0.000,100], d[0.187,1.106], g[1.083]
1/1 [==============================] - 0s 27ms/step
>4009, c[0.001,100], d[

KeyboardInterrupt: ignored

In [ ]:
# example of loading the classifier model and generating images
from numpy import expand_dims
from keras.models import load_model
# load the model
model_c = load_model('c_model_7200.h5')
# load the dataset
(trainX, trainy), (testX, testy) = load_data()
# expand to 3d, e.g. add channels
trainX = expand_dims(trainX, axis=-1)
testX = expand_dims(testX, axis=-1)
# convert from ints to floats
trainX = trainX.astype('float32')
testX = testX.astype('float32')
# scale from [0,255] to [-1,1]
trainX = (trainX - 127.5) / 127.5
testX = (testX - 127.5) / 127.5
# evaluate the model
_, train_acc = model_c.evaluate(trainX, trainy, verbose=0)
print('Train Accuracy: %.3f%%' % (train_acc * 100))
_, test_acc = model_c.evaluate(testX, testy, verbose=0)
print('Test Accuracy: %.3f%%' % (test_acc * 100))

In [ ]:
# generate random z vektor
import numpy as np

model_generator = load_model('g_model_7200.h5')

In [ ]:
z_sample = np.random.rand(1, 100)*2 - 1
X_pred = model_generator.predict(z_sample)
pyplot.imshow(X_pred[0, :, :, 0], cmap='gray_r')
most_likely_nb = np.argmax(model_c.predict(X_pred))
print(f"The most likely number is: {most_likely_nb}")